<a href="https://colab.research.google.com/github/DaraRahma536/TensorFlow-in-Action/blob/main/Chapter_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Chapter 3: Keras dan Retrieval Data di TensorFlow 2**

# **1. API Pembangunan Model di Keras**
---
Keras adalah submodul TensorFlow yang menyediakan API tingkat tinggi untuk membangun model ML dengan lebih mudah dan modular. Ada tiga API utama:

### **1. Sequential API**
* Paling sederhana, cocok untuk model linear (satu input → serangkaian layer → satu output).
* Kelemahan: tidak mendukung multiple input/output atau arsitektur kompleks.

Contoh implementasi model klasifikasi bunga Iris:

In [ ]:
model = Sequential([
    Dense(32, activation='relu', input_shape=(4,)),
    Dense(16, activation='relu'),
    Dense(3, activation='softmax')
])

### **2. Functional API**
* Lebih fleksibel, mendukung multiple input/output, layer paralel, dan branching.
* Cocok untuk model yang membutuhkan lebih dari satu input (misal: fitur raw + PCA).

Contoh:

In [ ]:
inp1 = Input(shape=(4,))
inp2 = Input(shape=(2,))
out1 = Dense(16, activation='relu')(inp1)
out2 = Dense(16, activation='relu')(inp2)
out = Concatenate(axis=1)([out1, out2])
...
model = Model(inputs=[inp1, inp2], outputs=out)

### **3. Sub-classing API**
* Paling fleksibel dan kompleks, memungkinkan pembuatan layer kustom dengan mendefinisikan class Python.
* Cocok ketika tidak ada layer bawaan yang sesuai (misal: layer dengan bias multiplikatif).

Contoh:

In [ ]:
class MulBiasDense(layers.Layer):
    def __init__(self, units=32, activation=None):
        super().__init__()
        self.units = units
        self.activation = activation

    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units))
        self.b = self.add_weight(shape=(self.units,))
        self.b_mul = self.add_weight(shape=(self.units,))

    def call(self, inputs):
        out = (tf.matmul(inputs, self.w) + self.b) * self.b_mul
        return layers.Activation(self.activation)(out)

# **2. Retrieval Data untuk Model TensorFlow/Keras**
---
Data adalah komponen kunci dalam pelatihan model. TensorFlow menyediakan beberapa cara untuk mengambil dan memproses data:

### **1. tf.data API**
* Paling fleksibel dan dapat dikustomisasi.
* Cocok untuk pipeline data kompleks dari sumber kustom (file CSV, gambar, dll).

Contoh pipeline untuk klasifikasi gambar bunga:

In [ ]:
# Baca CSV
csv_ds = tf.data.experimental.CsvDataset(...)
# Pisahkan filename dan label
frame_ds = csv_ds.map(lambda a,b: a)
label_ds = csv_ds.map(lambda a,b: b)
# Load dan preprocess gambar
image_ds = frame_ds.map(get_image)
# Gabungkan dan batch
data_ds = tf.data.Dataset.zip((image_ds, label_ds)).batch(5)

### **2. Keras DataGenerators**
Lebih sederhana dan cepat, cocok untuk data gambar dan time series.

```ImageDataGenerator``` untuk gambar, ```TimeseriesDataGenerator``` untuk data deret waktu.

Contoh:

In [ ]:
img_gen = ImageDataGenerator()
gen_iter = img_gen.flow_from_dataframe(
    dataframe=labels_df, directory=data_dir,
    x_col='file', y_col='label', batch_size=5
)

### **3. tensorflow-datasets Package**
* Paling mudah, tetapi terbatas pada dataset yang sudah tersedia.
* Cocok untuk eksperimen cepat dengan dataset populer (CIFAR10, IMDB, dll).

Contoh:

In [ ]:
import tensorflow_datasets as tfds
data, info = tfds.load("cifar10", with_info=True)
train_ds = data["train"].batch(16).map(format_data)

# **Kesimpulan Utama**
---
* Pilih API Keras berdasarkan kompleksitas model:  
-&nbsp;Sequential → model sederhana, linear.  
-&nbsp;Functional → model dengan multiple input/output.  
-&nbsp;Sub-classing → kebutuhan layer kustom.  

* Pilih metode retrieval data berdasarkan kebutuhan:  
-&nbsp;tf.data → pipeline kustom, fleksibel.  
-&nbsp;Keras DataGenerators → cepat untuk data standar.  
-&nbsp;tensorflow-datasets → dataset populer, mudah digunakan.  

* Reproducibility penting dalam ML → gunakan random seed tetap untuk hasil konsisten.

